In [356]:
import os
import pandas as pd

In [357]:
df = pd.read_csv('Data/spotify_millsongdata.csv')

In [358]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [359]:
df.tail()

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [360]:
df.shape

(57650, 4)

In [361]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [362]:
df = df.sample(6500).drop('link', axis=1, errors='ignore').reset_index(drop=True)


In [363]:
df.head()

,artist,song,text
0,Elvis Presley,Cattle Call,(Tex Owens) \r\nThe cattle are prowlin' and t...
1,Irving Berlin,You Can't Get A Man With A Gun,"Oh, my mother was frightened by a shotgun, the..."
2,Queens Of The Stone Age,Back To Dungaree High,"I didn't mean to turn you on, no way \r\nJust..."
3,Iggy Pop,1969,Well it's 1969 OK \r\nAll across the USA \r\...
4,Freestyle,Y,My baby I love you so much forever you and I ...


In [364]:
df["text"][0]

"(Tex Owens)  \r\nThe cattle are prowlin' and the coyotes are howlin'  \r\nWay out where the dogies bawl  \r\nWhere spurs are a-jinglin' a cowboy is singin'  \r\nThis lonesome cattle call  \r\nWoo-hoo woo-hoo hoo hoo  \r\nWoo-hoo woo-hoo hoo hoo  \r\nWoo-hoo woo-hoo hoo hoo hoo  \r\nWoo-hoo woo-hoo hoo  \r\nHe rides in the sun, till his days work is done  \r\nAnd he rounds up the cattle each fall  \r\nWoo-hoo woo-hoo hoo hoo  \r\nSingin' his cattle call.  \r\nFor hours he will ride on the range far and wide  \r\nWhen the night wind blows up a squall  \r\nHis heart is a feather in all kinds of weather  \r\nHe sings his cattle call  \r\nWoo-hoo woo-hoo hoo hoo  \r\nWoo-hoo woo-hoo hoo hoo  \r\nWoo-hoo woo-hoo hoo hoo hoo  \r\nWoo-hoo woo-hoo hoo  \r\nHe's brown as a berry from ridin' the prairie  \r\nAnd sings with an ol' western drawl  \r\nWoo-hoo woo-hoo hoo hoo  \r\nSingin' his cattle call\r\n\r\n"

In [365]:
df['text']=df['text'] = df['text'].str.lower().str.replace(r'^\w\s', ' ').replace(r'\n', '', regex=True)

In [366]:
df.head()

,artist,song,text
0,Elvis Presley,Cattle Call,(tex owens) \rthe cattle are prowlin' and the...
1,Irving Berlin,You Can't Get A Man With A Gun,"oh, my mother was frightened by a shotgun, the..."
2,Queens Of The Stone Age,Back To Dungaree High,"i didn't mean to turn you on, no way \rjust w..."
3,Iggy Pop,1969,well it's 1969 ok \rall across the usa \rit'...
4,Freestyle,Y,my baby i love you so much forever you and i ...


In [367]:
import nltk
from nltk.stem.porter import PorterStemmer

In [368]:
stemmer = PorterStemmer()

In [369]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a = [stemmer.stem(i) for i in token]
    return " ".join(a)

In [370]:
token("you are running faster than the others")

'you are run faster than the other'

In [371]:
df['text'] = df['text'].apply(token)

In [372]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [373]:
tfid = TfidfVectorizer( analyzer='word', stop_words='english') 

In [374]:
matrix = tfid.fit_transform(df['text'])

In [375]:
similar = cosine_similarity(matrix)

In [376]:
similar[0]

array([1.00000000e+00, 1.21549257e-02, 1.65227814e-03, ...,
       0.00000000e+00, 0.00000000e+00, 3.69057137e-04])

In [377]:
print(df['song'].sample(10).values)


['Fields Of People' "Nothin' To Lose" 'Bladerunner' 'Hooked On A Feeling'
 'Am I Evil?' 'Hymn To Her' 'Hand Of Providence' 'Born To Fight'
 'Beginning Of The End' 'Demons']


In [378]:
def recommender(song_name):
    matches = df[df['song'].str.lower() == song_name.lower()]
    if matches.empty:
        return f"❌ The song '{song_name}' was not found in the dataset."
    
    idx = matches.index[0]
    distances = sorted(list(enumerate(similar[idx])), key=lambda x: x[1], reverse=True)
    
    songs = []
    for i in distances[1:6]:  # skip itself
        songs.append(df.iloc[i[0]]['song'])
    return songs

In [379]:
recommender("If I Gave My Heart To You")

["Don't Go Breaking My Heart",
 'Last Christmas',
 'Give Your Heart A Break',
 'Love You By Heart',
 'You Put This Love In My Heart']